# Lecture: Data Management and Databases

In this lecture you will learn how to create and populate a relational database. After it is created, we will walk through the different strategies you can use to pull data from the database across its multiple tables. In this demonstration, we will use Python's sqlite3 library and pandas to write SQL calls.

## Connect to the Data and Create a Database

Mount Google Drive as the storage location.

In [0]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


Establish a database called lecture.db using the sqlite3 python module.

In [0]:
import sqlite3 #the sqlite3 library allows you to write SQL queries using Python

conn = sqlite3.connect('lecture.db') #establish the database and connect to it
cursor = conn.cursor() #create a cursor for the db. cursors allow you to retrieve, add, and remove individual database records when using sqlite3 functions.


This data is currently organized in seperate csv files. CSV (comma seperated value) files are tabular data structures (spreadsheets) created in a plain text format. We will need to combine these csv files into our database, but first these csv files need to be brought into Python using the Pandas pd.read_csv function.


In [0]:
import pandas as pd #the pandas library will handle the csvs

#create variables to store csvs that will make up the database
proceedure_events = pd.read_csv("/content/gdrive/My Drive/[YCMI_CBDS Summer Course] Data/mimic_iii/PROCEDURES_ICD.csv") #this command uses pandas to read in the csv file from its location on the mounted google drive
patients = pd.read_csv("/content/gdrive/My Drive/[YCMI_CBDS Summer Course] Data/mimic_iii/PATIENTS.csv") #this is repeated for each of the csvs we want to work with
icu_stays = pd.read_csv("/content/gdrive/My Drive/[YCMI_CBDS Summer Course] Data/mimic_iii/ICUSTAYS.csv")
microbiology_events = pd.read_csv("/content/gdrive/My Drive/[YCMI_CBDS Summer Course] Data/mimic_iii/MICROBIOLOGYEVENTS.csv")
admissions = pd.read_csv("/content/gdrive/My Drive/[YCMI_CBDS Summer Course] Data/mimic_iii/ADMISSIONS.csv")
proceedure_icd = pd.read_csv("/content/gdrive/My Drive/[YCMI_CBDS Summer Course] Data/mimic_iii/D_ICD_PROCEDURES.csv") 

#display the .head() of each csv to confirm it was imported properly
print(proceedure_events)
print(patients.head())
print(icu_stays.head())
print(microbiology_events.head())
print(admissions.head())
print(proceedure_icd.head())

     row_id  subject_id  hadm_id  seq_num  icd9_code
0      3994       10114   167957        1       3605
1      3995       10114   167957        2       3722
2      3996       10114   167957        3       8856
3      3997       10114   167957        4       9920
4      3998       10114   167957        5       9671
..      ...         ...      ...      ...        ...
501  230170       41976   179418        2       3893
502  235193       41976   155297        1       3893
503  235194       41976   155297        2        966
504  238588       41976   125013        1       3893
505  239961       41976   151798        1        966

[506 rows x 5 columns]
   row_id  subject_id  ...              dod_ssn expire_flag
0    9467       10006  ...  2165-08-12 00:00:00           1
1    9472       10011  ...                  NaN           1
2    9474       10013  ...  2125-10-07 00:00:00           1
3    9478       10017  ...  2152-09-12 00:00:00           1
4    9479       10019  ...  2163-05-15 0

In [0]:
#the following creates a list containing all of the dataframes we have read into our python project
mimic_csv_list = [proceedure_events, patients, icu_stays, microbiology_events, admissions, proceedure_icd]

#the following for loop prints every column name for each of our dataframes contained in the list created above. The empty print statement results in a blank space between 
#the lists of column names
for csv in mimic_csv_list: 
    for column_name in csv.columns:
      print(column_name)
    print()

row_id
subject_id
hadm_id
seq_num
icd9_code

row_id
subject_id
gender
dob
dod
dod_hosp
dod_ssn
expire_flag

row_id
subject_id
hadm_id
icustay_id
dbsource
first_careunit
last_careunit
first_wardid
last_wardid
intime
outtime
los

row_id
subject_id
hadm_id
chartdate
charttime
spec_itemid
spec_type_desc
org_itemid
org_name
isolate_num
ab_itemid
ab_name
dilution_text
dilution_comparison
dilution_value
interpretation

row_id
subject_id
hadm_id
admittime
dischtime
deathtime
admission_type
admission_location
discharge_location
insurance
language
religion
marital_status
ethnicity
edregtime
edouttime
diagnosis
hospital_expire_flag
has_chartevents_data

row_id
icd9_code
short_title
long_title



Next, the pandas .to_sql method statements below will add the data from these six dataframes to lecture.db database. Each dataframe will become a seperate table within the database. 

In [0]:
proceedure_events.to_sql("proceedure_events", conn, if_exists="replace", index = False) #"proceedure_events" becomes the name of the table in the database. 
patients.to_sql("patients", conn, if_exists="replace", index = False) #conn is the cursor that points to the lecture.db.
icu_stays.to_sql("icu_stays", conn, if_exists="replace", index = False) #"index = false" indicates that there is no row headers or an index column in the dataframe.
microbiology_events.to_sql("microbiology_events", conn, if_exists="replace", index = False)
admissions.to_sql("admissions", conn, if_exists="replace", index = False)
proceedure_icd.to_sql("proceedure_icd", conn, if_exists = "replace", index = False)

## Exploring the MIMIC III Demo Database

### Quality Checking Considerations
* Who created this dataset originally?
* Does this dataset have any corrisponding documentation? 
* Are there any limitations on how this database can be used?

Find information on the MIMIC III database here: https://mimic.physionet.org/gettingstarted/demo/


### Data Management Considerations

Take time to look through the documentation associated with the mimic iii database. Understanding what data exists in a database is vital in order to form queries.

* Where is the data stored and in what format?
  * The csvs are stored in Google Drive, and mounted to our project a command used above
* What is included in the database? What is not included that you might expect to be included?
  * The database schema can be found here: https://mit-lcp.github.io/mimic-schema-spy/
* What is the structure of this database?
  * What are the names of each of the tables included in the database?
  * What data do the tables contain?
  * Each of the tables in this database has documentation about its contents: https://mimic.physionet.org/mimictables/admissions/


##### Return the names of each of the tables in the database

read_sql_query is a Pandas function that allows you to create SQL statements in Python. The SQL statement exists between quotation marks. 

The statement below selects the name column from the database metadata table, which is created automatically and is called sqlite_master. This allows us to see all of the table names in a database. If you did not have available database documentation, this is a strategy you could use to learn about which tables exist within a database.


In [0]:
pd.read_sql_query('SELECT name FROM sqlite_master WHERE type = "table";', conn) #pd.read_sql_query is the function, the first argument in the function is the SQL script, and the second
#argument is the conn variable, whichs connects the function to the database.


,name
0,proceedure_events
1,patients
2,icu_stays
3,microbiology_events
4,admissions
5,proceedure_icd


##### Return the data types of each column within a table

Next, here are two ways to do the same task. The first uses the pandas .read_sql method, and the second uses a method from the sqlite3 library .fetchall.

Both use a cursor / conn method to point to and connect to the lecture.db database in order to look at the metadata contained in the sqlite_master file assocateed with the individual database table, "icu_stays".

Each provides the same data output, but in differing views. I point this out because data output can determine which function you may or may not want to use. In the output below, the sqlite3 output is more clear (because it is not truncated) than the pandas output. 

However, in this work book, we will rely on the pandas functions because when performing data subset queries, the output is easier to read. The code below results in a return that provides metadata about the data contained in the tables of the database.You can see that row_id is an integer, subject_id is an integer, first_careunit is text. So, we see the column name as well as the data type held within that column. 

In [0]:
print("pandas output:", pd.read_sql_query('SELECT sql FROM sqlite_master WHERE type = "table" AND tbl_name = "icu_stays";', conn)) #this 
print() #prints a blank line
cursor.execute("select sql from sqlite_master where type = 'table' and tbl_name = 'icu_stays';") #establishes a sql query in quotation marks and connect this to the cursor 
print("sqlite3 output:", cursor.fetchall()) #this prints the result of what is gathered from the cursor execution 





pandas output:                                                  sql
0  CREATE TABLE "icu_stays" (\n"row_id" INTEGER,\...

sqlite3 output: [('CREATE TABLE "icu_stays" (\n"row_id" INTEGER,\n  "subject_id" INTEGER,\n  "hadm_id" INTEGER,\n  "icustay_id" INTEGER,\n  "dbsource" TEXT,\n  "first_careunit" TEXT,\n  "last_careunit" TEXT,\n  "first_wardid" INTEGER,\n  "last_wardid" INTEGER,\n  "intime" TEXT,\n  "outtime" TEXT,\n  "los" REAL\n)',)]


## Pulling Data from the MIMIC III Demo Database (Single Table)



##### Pull all data from a single table using FROM and SELECT

The script below utilizes the sqlite3 library. This library attaches the SQL query to the database cursor with the execute function. The SQL function used is
"SELECT * FROM patients;".

"*" indicates that we want all of the records. "*" acts as a placeholder for "all". "patients" is the patient table name that was originally our csv files that was added to the lecture.db database.

In order to  read the output of this database call, which is stored in the variable "result." We would need to put result into a for loop. To avoid this extra step and because pandas output is easier to read in these instances, we will use the pandas read.sql function to see the results of the sql queries from here on out.

Briefly showing two different libraries that function similarly is a good reminder that not only does Python have libraries that perform the same tasks - but that these libraries think interact with and out put data in differing ways.


In [0]:
#print(pd.read_sql_query('SELECT * FROM PATIENTS', conn))
cursor.execute("SELECT * FROM patients;") #SQL code linked to the cursor
result = cursor.fetchall() 
for r in result:
  print(r)


(9467, 10006, 'F', '2094-03-05 00:00:00', '2165-08-12 00:00:00', '2165-08-12 00:00:00', '2165-08-12 00:00:00', 1)
(9472, 10011, 'F', '2090-06-05 00:00:00', '2126-08-28 00:00:00', '2126-08-28 00:00:00', None, 1)
(9474, 10013, 'F', '2038-09-03 00:00:00', '2125-10-07 00:00:00', '2125-10-07 00:00:00', '2125-10-07 00:00:00', 1)
(9478, 10017, 'F', '2075-09-21 00:00:00', '2152-09-12 00:00:00', None, '2152-09-12 00:00:00', 1)
(9479, 10019, 'M', '2114-06-20 00:00:00', '2163-05-15 00:00:00', '2163-05-15 00:00:00', '2163-05-15 00:00:00', 1)
(9486, 10026, 'F', '1895-05-17 00:00:00', '2195-11-24 00:00:00', None, '2195-11-24 00:00:00', 1)
(9487, 10027, 'F', '2108-01-15 00:00:00', '2190-09-14 00:00:00', None, '2190-09-14 00:00:00', 1)
(9489, 10029, 'M', '2061-04-10 00:00:00', '2140-09-21 00:00:00', None, '2140-09-21 00:00:00', 1)
(9491, 10032, 'M', '2050-03-29 00:00:00', '2138-05-21 00:00:00', '2138-05-21 00:00:00', '2138-05-21 00:00:00', 1)
(9492, 10033, 'F', '2051-04-21 00:00:00', '2133-09-09 00:00

##### Pull specific columns from a table using FROM and SELECT

This is a pandas based SQL query for comparison. Though in this query, specific columns (subject_id, gender, dob, and dod_hosp) are requested from the patients table, rather than all of the columns from this table. 

In [0]:
pd.read_sql_query(
    '''SELECT subject_id, gender, dob, dod_hosp 
    FROM patients;''',
    conn) # conn, here, connects this query to the database rather than the cursor. 
    #The cursor is built in to the pandas function, so we do not need the extra specification, as is needed with the sqlite3 library.

,subject_id,gender,dob,dod_hosp
0,10006,F,2094-03-05 00:00:00,2165-08-12 00:00:00
1,10011,F,2090-06-05 00:00:00,2126-08-28 00:00:00
2,10013,F,2038-09-03 00:00:00,2125-10-07 00:00:00
3,10017,F,2075-09-21 00:00:00,None
4,10019,M,2114-06-20 00:00:00,2163-05-15 00:00:00
...,...,...,...,...
95,44083,M,2057-11-15 00:00:00,2114-02-20 00:00:00
96,44154,M,1878-05-14 00:00:00,2178-05-15 00:00:00
97,44212,F,2078-06-16 00:00:00,None
98,44222,M,2107-06-27 00:00:00,2182-08-03 00:00:00


##### Pull data based on specified parameters from a single table using the WHERE clause

WHERE clause operators include:


*   = : equal
*   \> : greater than
*   < : less than
*   \>= : greater than or equal
*   <= : less than or equal
*   <> or != : not equal
*   BETWEEN : between a certain range
*   LIKE : search for a pattern (%pattern string%)
*   IN : speficic value pattern

These allow you to call data based on a value. Maybe you want a particilar time or age range, or maybe you want to include only a certain insurance type.




##### This query will return the columns subject_id, insurance, diagnosis, and admission_location from the admissions table, including only rows that have SEPSIS as the value within the diagnosis column.

In [0]:
pd.read_sql_query('''SELECT subject_id, insurance, diagnosis, admission_location FROM admissions WHERE diagnosis = "SEPSIS";''',conn)

,subject_id,insurance,diagnosis,admission_location
0,10006,Medicare,SEPSIS,EMERGENCY ROOM ADMIT
1,10013,Medicare,SEPSIS,TRANSFER FROM HOSP/EXTRAM
2,10036,Medicare,SEPSIS,EMERGENCY ROOM ADMIT
3,10056,Medicare,SEPSIS,TRANSFER FROM HOSP/EXTRAM
4,40601,Private,SEPSIS,EMERGENCY ROOM ADMIT
5,41976,Medicare,SEPSIS,EMERGENCY ROOM ADMIT
6,41976,Medicare,SEPSIS,CLINIC REFERRAL/PREMATURE
7,41976,Medicare,SEPSIS,CLINIC REFERRAL/PREMATURE
8,41976,Medicare,SEPSIS,CLINIC REFERRAL/PREMATURE
9,41976,Medicare,SEPSIS,CLINIC REFERRAL/PREMATURE


##### This query will return the same columns as above, but with less exact parameters

Using LIKE instead of =, and adding "%" around our string SEPSIS, allows us to see SEPSIS + other strings.

In [0]:
pd.read_sql_query('SELECT subject_id, insurance, diagnosis, admission_location FROM admissions WHERE diagnosis LIKE "%SEPSIS%";',conn)

,subject_id,insurance,diagnosis,admission_location
0,10006,Medicare,SEPSIS,EMERGENCY ROOM ADMIT
1,10013,Medicare,SEPSIS,TRANSFER FROM HOSP/EXTRAM
2,10036,Medicare,SEPSIS,EMERGENCY ROOM ADMIT
3,10056,Medicare,SEPSIS,TRANSFER FROM HOSP/EXTRAM
4,10088,Medicare,SEPSIS;PNEUMONIA;TELEMETRY,EMERGENCY ROOM ADMIT
5,10088,Private,UROSEPSIS,TRANSFER FROM HOSP/EXTRAM
6,10094,Medicare,SEPSIS;TELEMETRY,EMERGENCY ROOM ADMIT
7,40601,Private,SEPSIS,EMERGENCY ROOM ADMIT
8,41976,Medicare,SEPSIS,EMERGENCY ROOM ADMIT
9,41976,Medicare,SEPSIS,CLINIC REFERRAL/PREMATURE


##### Pull data based on multiple parameters

You can multiple conditional parameters by adding AND or OR between them. The query below returns specified columns from the microbiology_events table, where rows contain either BLOOD CULTURE as the spec_type_desc AND a dilution_value greater than or equal to 6.

In [0]:
pd.read_sql_query('SELECT row_id, spec_itemid, dilution_value, org_name FROM microbiology_events WHERE spec_type_desc = "BLOOD CULTURE" AND dilution_value >= 6;', conn)

,row_id,spec_itemid,dilution_value,org_name
0,134700,70012,8.0,"STAPHYLOCOCCUS, COAGULASE NEGATIVE"
1,134802,70012,8.0,GRAM POSITIVE COCCUS(COCCI)
2,134885,70012,32.0,ESCHERICHIA COLI
3,134887,70012,8.0,ESCHERICHIA COLI
4,134888,70012,16.0,ESCHERICHIA COLI
5,134896,70012,8.0,ENTEROCOCCUS SP.
6,134897,70012,16.0,ENTEROCOCCUS SP.
7,135478,70012,8.0,"STAPHYLOCOCCUS, COAGULASE NEGATIVE"
8,135479,70012,16.0,"STAPHYLOCOCCUS, COAGULASE NEGATIVE"
9,135484,70012,8.0,"STAPHYLOCOCCUS, COAGULASE NEGATIVE"


##### Find distinct data occurances with DISTINCT

This will return unique values from a column. This can be a quick way to see all of the categories represented in a dataset.

In [0]:
pd.read_sql_query('SELECT DISTINCT admission_type FROM admissions;', conn)
pd.read_sql_query('SELECT DISTINCT admission_location FROM admissions;', conn)

,admission_location
0,EMERGENCY ROOM ADMIT
1,TRANSFER FROM HOSP/EXTRAM
2,PHYS REFERRAL/NORMAL DELI
3,CLINIC REFERRAL/PREMATURE
4,TRANSFER FROM SKILLED NUR


##### Show ordered resulting data

Now that the SQL queries are getting longer, I will start breaking them up into multiple lines by enclosing the statement in 3 single quotes (seen below). 

The query below returns selected columns from the admissions table oreded by admission type (ascending), and admission_location (decending).

In [0]:
pd.read_sql_query('''SELECT subject_id, admission_type, admission_location, admittime 
FROM admissions 
ORDER BY admission_type ASC, admission_location DESC;''', conn)

,subject_id,admission_type,admission_location,admittime
0,42231,ELECTIVE,PHYS REFERRAL/NORMAL DELI,2102-08-29 07:15:00
1,44083,ELECTIVE,PHYS REFERRAL/NORMAL DELI,2112-05-04 08:00:00
2,10035,ELECTIVE,PHYS REFERRAL/NORMAL DELI,2129-03-03 16:06:00
3,43927,ELECTIVE,PHYS REFERRAL/NORMAL DELI,2175-10-02 12:30:00
4,10065,ELECTIVE,PHYS REFERRAL/NORMAL DELI,2189-09-08 07:15:00
...,...,...,...,...
124,41976,EMERGENCY,CLINIC REFERRAL/PREMATURE,2202-02-15 19:01:00
125,41976,EMERGENCY,CLINIC REFERRAL/PREMATURE,2202-09-16 21:56:00
126,41976,EMERGENCY,CLINIC REFERRAL/PREMATURE,2202-10-03 01:45:00
127,10112,URGENT,TRANSFER FROM HOSP/EXTRAM,2148-01-13 22:32:00


## Pulling Data from the MIMIC III Demo Database (Multiple Tables), i.e. Joining

Joins in SQL are used to combine columns from two or more tables, based on shared columns containing elements to link (or join) by.

View an interactive visualization of join components here: https://joins.spathon.com/

This next query will return subject_ids from the patients table (patients.subject_id), date of birth, insurance, marital_status FROM the patients table and the admissions table - JOINing these tables together by the subject_id column.

Because subject_id appears in both the paitients table and the admissions table, we can JOIN these tables together. Also, because subject_id appears in both tables, we need to specify which column we want SELECTed by indicating tablename.subject_id (patients.subject_id below). 

The ON specification in the query below establishes this linking value.

In [0]:
#Pull together subject date of birth (from the patients table) with insurance and marital statuses (from admissions table)
pd.read_sql_query('''SELECT patients.subject_id, dob, insurance, marital_status 
FROM patients 
INNER JOIN admissions 
ON patients.subject_id = admissions.subject_id;''', conn)

,subject_id,dob,insurance,marital_status
0,10006,2094-03-05 00:00:00,Medicare,SEPARATED
1,10011,2090-06-05 00:00:00,Private,SINGLE
2,10013,2038-09-03 00:00:00,Medicare,None
3,10017,2075-09-21 00:00:00,Medicare,DIVORCED
4,10019,2114-06-20 00:00:00,Medicare,DIVORCED
...,...,...,...,...
124,44083,2057-11-15 00:00:00,Private,SINGLE
125,44154,1878-05-14 00:00:00,Medicare,MARRIED
126,44212,2078-06-16 00:00:00,Medicare,SINGLE
127,44222,2107-06-27 00:00:00,Medicare,SINGLE


##### Conditional parameters can still be applied when using JOINed databases

The statement below is almost exactly the same as the statement above. The only difference is that only subjects who are denoted as WIDOWED or SINGLE in the marital_status column are included.

In [0]:
#Refine the query above by only including subjects who are either single or widowed
pd.read_sql_query('''SELECT patients.subject_id, marital_status, dob, insurance 
FROM patients INNER JOIN admissions 
ON patients.subject_id = admissions.subject_id 
WHERE marital_status = "WIDOWED" OR marital_status = "SINGLE";''', conn)

,subject_id,marital_status,dob,insurance
0,10011,SINGLE,2090-06-05 00:00:00,Private
1,10032,WIDOWED,2050-03-29 00:00:00,Medicare
2,10038,WIDOWED,2056-01-27 00:00:00,Medicare
3,10046,SINGLE,2141-03-15 00:00:00,Medicaid
4,10061,WIDOWED,2031-05-19 00:00:00,Medicare
5,10065,SINGLE,2111-07-18 00:00:00,Private
6,10090,SINGLE,2096-02-27 00:00:00,Private
7,10093,WIDOWED,2053-09-08 00:00:00,Medicare
8,10111,WIDOWED,2097-01-16 00:00:00,Medicare
9,10117,SINGLE,2072-05-05 00:00:00,Private


##### Left Joins: Add relevant columns to your primary (left) spreadsheet from secondary spreadsheets, while keeping rows in the primary spreadsheet which have no linked values from the secondary spreadsheets.

The example below contains two LEFT JOINs. Since proceedures are listed individually in the proceedure_events table (each subject may have 0, 1 or more proceedures per admission), and actual proceedure titles are listed in the proceedures_icd table, in order to work across all three tables, you will need to perform the two joins shown below.

The result below pulls together data from the admissions tables and the proceedures_icd table, using the proceedure_events table as a sort of cross-walk between the tables.

In [0]:
#perform a left join to link proceedure names in the proceedure_icds to the icd codes present 
#in the proceedure_events table, and present these proceedure names with specific admission instances.

pd.read_sql_query('''SELECT admissions.subject_id, discharge_location, proceedure_icd.short_title 
FROM admissions 
LEFT JOIN proceedure_events ON admissions.subject_id = proceedure_events.subject_id
LEFT JOIN proceedure_icd ON proceedure_events.icd9_code = proceedure_icd.icd9_code;''', conn)

,subject_id,discharge_location,short_title
0,10006,HOME HEALTH CARE,Injection oxazolidinone
1,10006,HOME HEALTH CARE,Venous cath NEC
2,10006,HOME HEALTH CARE,Ven cath renal dialysis
3,10006,HOME HEALTH CARE,Hemodialysis
4,10006,HOME HEALTH CARE,Percu abdominal drainage
...,...,...,...
1331,44212,REHAB/DISTINCT PART HOSP,Cont inv mec ven 96+ hrs
1332,44222,HOME,None
1333,44228,HOME HEALTH CARE,Venous cath NEC
1334,44228,HOME HEALTH CARE,Venous cath NEC


##### Let's look at this a bit more closely

First, by viewing the proceedure_events table and observing where the icd9_codes, icd short_titles, and discharge appear across the dataframes.

In [0]:
print(proceedure_events.head())
print(proceedure_icd.head())
print(admissions.head())

     icd9_code
0         3605
1         3722
2         8856
3         9920
4         9671
..         ...
501       3893
502       3893
503        966
504       3893
505        966

[506 rows x 1 columns]
   row_id  subject_id  hadm_id  seq_num  icd9_code
0    3994       10114   167957        1       3605
1    3995       10114   167957        2       3722
2    3996       10114   167957        3       8856
3    3997       10114   167957        4       9920
4    3998       10114   167957        5       9671


##### Summarizing data results using COUNT and GROUP BY

You can use a COUNT function to create a summary read out if you also determine what you would like to GROUP BY. Below, you can see how if you GROUP BY subject_id, you are then able to COUNT the number of icd9_codes per each of these subject_ids within the proceedure_events column. This essentially tells us that for each subject indexed in the, how many proceedures the subject has undergone.

Since COUNT is now one of our columns in this data pull, we can utilize apply a conditional statement to this column. HAVING allows us to filter on this summary column. Below, only patientes who have had more than 15 proceedures are included in the final results. 




In [0]:
pd.read_sql_query('''SELECT subject_id, COUNT(icd9_code) 
FROM proceedure_events
GROUP BY subject_id
HAVING COUNT(icd9_code) > 15 ''', conn)

,subject_id,COUNT(icd9_code)
0,10126,22
1,10127,19
2,40310,18
3,41976,49
4,42075,16


In this last example, you can see how you can bring all of the SQL query elements together to built out highly customizable database queries. Below, this query provides the names of proceedure events that have occured in the database more than 10 times by bringing together the proceedure_icd and proceedure_events tables with a left join. 

In [0]:
pd.read_sql_query('''SELECT proceedure_icd.short_title, proceedure_events.icd9_code, COUNT(proceedure_events.icd9_code) 
FROM proceedure_events
LEFT JOIN proceedure_icd on proceedure_events.icd9_code = proceedure_icd.icd9_code
GROUP BY proceedure_events.icd9_code
HAVING COUNT(proceedure_events.icd9_code) > 10;''', conn)

,short_title,icd9_code,COUNT(proceedure_events.icd9_code)
0,Entral infus nutrit sub,966,32
1,Venous cath NEC,3893,56
2,Percu abdominal drainage,5491,12
3,Insert endotracheal tube,9604,22
4,Cont inv mec ven <96 hrs,9671,21
5,Cont inv mec ven 96+ hrs,9672,19
6,Packed cell transfusion,9904,30
7,Parent infus nutrit sub,9915,14


## Overview: Bringing SQL Query Elements Together

In order to build out more complex single queries in SQL, you will follow the structure below. To generalize the theme of this structure: SQL queries startby pointing at relevant columns and tables (lines 1 through 4 below), specifying (cell) values of interest (line 5 below), and finally choosing how the data is returned (lines 6 through 7).

As we saw in the SQL in previous examples, lines 3 through 7 in the code snippet below are optional and they are included in a query, they are also expandable. 

```
1. SELECT column_name
2. FROM table1_name
3.     JOIN table2_name
4.       ON table1_name.linking_column_name = table2_name.linking_column_name
5.     WHERE contraint_expression
6.     GROUP BY column_name
7.     ORDER BY column_name ASC/DESC
```

As you work through the exercises for this session, revisit the database documentation links provided in this lecture workbook. Think about other questions you can ask of the mimic iii database and piece together a SQL query that will result in you retreving data that will enabe you to answer your question.
